In [1]:
#run this code to download dataset directly from URL

#for c in ['sports', 'religion', 'local', 'international', 'economy', 'culture']:
#    r = requests.get(
#        'https://sourceforge.net/projects/kalimat/files/kalimat/document-collection/articles-'+c+'.zip/download')
#    z = zipfile.ZipFile(io.BytesIO(r.content))
#    z.extractall('./newData/'+c)

# ----replace "Data" with "newData"


## Language Modeling
* dataset= https://sourceforge.net/projects/kalimat/files/kalimat/document-collection/

In [1]:
import os
#for performance measure:
import pandas as pd
import nltk
import util as ut



Bi-grams

In [3]:

test_text1 = "تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون والاذاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو"
err_text1 = "تنطلق اليوم الدورة البرامجية الجديدة للتليفزيون و الاشاعة و برنامج الشباب و التي تستمر طوال اشهر ابريل و مايو و يونيو"

test_text2 = "حقق منتخبنا الوطني الاول لكرة القدم كل اهدافه المطلوبة"
err_text2 = "حقق منتخبنا الوطني الاول لكرة القسم كل اهدافه المطلوبة"
err_text2 = "منتخبنا يتألق ويهم الشباك"


* add cleaning, lemmatizing + removing stop words

In [4]:
#lemmatization
#! pip install qalsadi
import qalsadi.lemmatizer
lemmer = qalsadi.lemmatizer.Lemmatizer()
def lemmatize(text):
    t=lemmer.lemmatize_text(text)
    s = ut.stringify(t)
    return s


* sortir de la boucle si on retrouve un ngram tres proche (inversion/suppression/ajout d'un caratere) :

In [5]:
# return ngram that don't exist in corpustoken or exist with minor change (typography mistake)
def OneSubstitution(word, word2):# return true if exists one and only onesubstituion error
    if len(word)==len(word2):
        l = []
        for i in range(len(word)):
            if word[i] == word2[i]:
                l.append(0)
            else:
                l.append(1)
        return (sum(l) == 1)
    else:
        return(False)


def OneInversion(word, word2):
    if len(word) == len(word2):  # returns true if exists one and only inversion error
        l1 = []
        l2 = []
        for i in range(len(word)):
            if word[i] == word2[i]:
                l1.append(0)
            else:
                l1.append(1)
                l2.append(word[i])
                l2.append(word2[i])
        if sum(l1) == 2:
          return(l2[0] == l2[3] and l2[1] == l2[2])
        else:
            return(False)
    else:
        return(False)


def AddSubErr(word1, word2):  # ajout/suppression d'un seul caracter
    w1 = set(word1)
    w2 = set(word2)
    maxi=max(len(w1-w2),len(w2-w1))
    if maxi != 1:
        return(False)
    else :
        if len(w1)-len(w2) == 1:
            lw1=list(word1)
            lw1.remove(''.join(w1-w2))
            s = ''.join(str(x) for x in lw1)
            return(s==word2)
        elif len(w1)-len(w2) == -1:
            lw2 = list(word2)
            lw2.remove(''.join(w2-w1))
            s = ''.join(str(x) for x in lw2)
            return(s == word1)
        else:
            return(False)
def lookslike(words, corpustoken):
    for word in words:
        for c in corpustoken:
            if AddSubErr(word,c): 
                return(word)
            elif OneInversion(word,c) or OneSubstitution(word,c) : #inversion / substitution
                return(word)
    return('')

def CheckNGramInCorpus(texttoken, corpustoken):
    ngram = [g for g in texttoken if g not in corpustoken]
    return (ngram)


In [6]:
def WrongNGram(text, n=2):
    text = ut.very_clean(text)
    #text = lemmatize(text)
    token = ut.get_all_ngrams(text, nrange=n)
    ngram_i = []
    err = ''
    for directory in os.listdir("../Data"):
        for filename in os.listdir("../Data/"+directory):
            f = open("../Data/" + directory+"/"+filename, "r", encoding="utf8")
            g = ut.very_clean(f.read().replace('\n', ' '))
            #g = lemmatize(g)
            ngram_i = ut.get_all_ngrams(g, nrange=n)
            #print(token)
            #print(ngram_i)
            token = CheckNGramInCorpus(token, ngram_i)
            err = lookslike(token,ngram_i)
            #if len(err) !=0 :
                #print("potential error in: ",err) 
            if len(token) == 0:
                #print(token ,'no error')
                return(token)
            f.close()
    #print("potential error in :", err)
    #print("error in : \n ",token)
    return(token)


## Execute

In [7]:
def mispelled_word (text,n=2): #sans considerer lemmatizationb
    l = WrongNGram(text)
    #print(l)
    if len(l)>1:
        l2 = l[0].split("_")
        #print("mispelled word is : ")
        return(l2[-1])
    elif len(l) == 1 : #mistake in first or last word
        #print("the error is either first or last word")
        return(l)
    else :
        #print("no error in this sentence")
        return(None)


In [8]:
mispelled_word(err_text2)

'ويهم'

# Evaluation du modele

In [9]:
df = pd.read_csv("../testdata.csv", dtype='string')


* y vs y_predit

In [10]:
test = [ c for c in df.sentence]
y_ngram_pred = [mispelled_word(c) for c in test]
y_pred = []
for i in y_ngram_pred:
    if type(i) == str:
        y_pred.append(i)
    else:
        y_pred.append(i[0])
y = [y.lstrip().rstrip() for y in df.wrong_word]


In [11]:
#print(y)
#print(y_pred)

* precision & recall

In [12]:
def precision(y, y_pred):
    c = 0  # nbre de mots correctement étiquetés
    for i in range(len(y)):
        if y[i] == y_pred[i]:
            c += 1

    p = c / len(y)
    return(p)


In [13]:
prec = precision(y, y_pred)
print("language modelisong precision = ",prec)


language modelisong precision =  0.5454545454545454
